# Mistplay Fraud Demo – Batch Inference

This notebook scores recent accounts using the registered model and writes risk scores to a Delta table for analyst review.

In [ ]:
from databricks.feature_store import FeatureStoreClient
from pyspark.sql import functions as F

DB_NAME = "mistplay_fraud_demo"
MODEL_NAME = "mistplay_fraud_model"

fs = FeatureStoreClient()

account_device = spark.table(f"{DB_NAME}.account_device")
events = spark.table(f"{DB_NAME}.events")

# Simulate a 1-day lookback of active accounts
recent_accounts = (
    events.filter(F.col("event_ts") >= F.expr("current_timestamp() - interval 1 day"))
    .select("account_id")
    .distinct()
)

scoring_input = recent_accounts.join(account_device, on="account_id", how="left")

model_uri = f"models:/{MODEL_NAME}/Latest"

predictions = fs.score_batch(model_uri=model_uri, df=scoring_input)

scored = (
    predictions
    .withColumnRenamed("prediction", "fraud_score")
    .withColumn(
        "risk_bucket",
        F.when(F.col("fraud_score") >= 0.8, F.lit("high"))
        .when(F.col("fraud_score") >= 0.5, F.lit("medium"))
        .otherwise(F.lit("low")),
    )
    .withColumn("scored_at", F.current_timestamp())
)

scored.write.mode("overwrite").saveAsTable(f"{DB_NAME}.fraud_scores_batch")

print("Batch scores saved to:")
print(f"- {DB_NAME}.fraud_scores_batch")